In [1]:
!ls ../data/uniprot

embeddings_vis.png	     processed_03.tsv
encoder_training_data.tsv    protein_sequences.txt
encoder_validation_data.tsv  raw_dataset_01.tsv
gene_ontology		     raw_testset.tsv
go_terms.txt		     sequences.txt
NOTES.txt		     uniprotkb_AND_reviewed_true_2023_12_23.tsv
processed_01.tsv	     uniprotkb_sequence_and_go_terms.tsv
processed_02.tsv	     uniprotkb_sequence_AND_superkingdom.tsv


In [4]:
import pandas as pd
from tqdm.notebook import tqdm

In [5]:
sequence_terms_df = pd.read_csv("../data/uniprot/uniprotkb_sequence_and_go_terms.tsv", sep="\t")

sequence_terms_df.head()

,Entry,Reviewed,Gene Ontology IDs,Sequence
0,A0A087X1C5,reviewed,GO:0005506; GO:0005737; GO:0005739; GO:0006805...,MGLEALVPLAMIVAIFLLLVDLMHRHQRWAARYPPGPLPLPGLGNL...
1,A0A0B4J2F0,reviewed,GO:0005741; GO:0006986; GO:1900101,MFRRLTFAQLLFATVLGIAGGVYIFQPVFEQYAKDQKELKEKMQLV...
2,A0A0B4J2F2,reviewed,GO:0000287; GO:0004674; GO:0005524; GO:0005634...,MVIMSEFSADPAGQGQGQQKPLRVGFYDIERTLGKGNFAVVKLARH...
3,A0A0C5B5G6,reviewed,GO:0001649; GO:0003677; GO:0005615; GO:0005634...,MRWQEMGYIFYPRKLR
4,A0A0K2S4Q6,reviewed,GO:0004888; GO:0005576; GO:0005886; GO:0030593,MTQRAGAAMLPSALLLLCVPGCLTVSGPSTVMGAVGESLSVQCRYE...


In [6]:
term_name_df = pd.read_csv("../data/go_term_name_dataset.tsv.csv", sep="\t")

term_name_df.head()

,id,value
0,GO:0000967,biological_process rRNA 5'-end processing
1,GO:0001646,molecular_function cAMP receptor activity
2,GO:0001765,biological_process membrane raft assembly
3,GO:0001916,biological_process positive regulation of T ce...
4,GO:0001993,biological_process regulation of systemic arte...


In [10]:
term_name_dict = {}

In [11]:
for idx, row in tqdm(term_name_df.iterrows(), total=len(term_name_df)):
    term_name_dict[row['id']] = row['value']

  0%|          | 0/42769 [00:00<?, ?it/s]

In [12]:
sequence_terms_df["Len"] = sequence_terms_df["Sequence"].map(len)

In [13]:
sequence_terms_df1 = sequence_terms_df[sequence_terms_df["Len"] <= 512]

sequence_terms_df1.head()

,Entry,Reviewed,Gene Ontology IDs,Sequence,Len
1,A0A0B4J2F0,reviewed,GO:0005741; GO:0006986; GO:1900101,MFRRLTFAQLLFATVLGIAGGVYIFQPVFEQYAKDQKELKEKMQLV...,54
3,A0A0C5B5G6,reviewed,GO:0001649; GO:0003677; GO:0005615; GO:0005634...,MRWQEMGYIFYPRKLR,16
4,A0A0K2S4Q6,reviewed,GO:0004888; GO:0005576; GO:0005886; GO:0030593,MTQRAGAAMLPSALLLLCVPGCLTVSGPSTVMGAVGESLSVQCRYE...,201
5,A0A0U1RRE5,reviewed,GO:0000932; GO:0000956; GO:0006397; GO:0010607,MGDQPCASGRSTLPPGNAREAKPPKKRCLLAPRWDYPEGTPNGGST...,68
10,A0AVI4,reviewed,GO:0000209; GO:0005783; GO:0005789; GO:0006986...,MDSPEVTFTLAYLVFAVCFVFTPNEFHAAGLTVQNLLSGWLGSEDA...,362


In [14]:
sequence_terms_df1.shape

(12663, 5)

In [24]:
sequence_terms_df2 = sequence_terms_df1.dropna()

sequence_terms_df2.shape

(11764, 5)

In [29]:
dataset = {
    "Sequence": [],
    "Annotation": []
}

In [30]:
for idx, row in tqdm(sequence_terms_df2.iterrows(), total=len(sequence_terms_df2)):
    terms = list(map(lambda obj: obj.strip(), row["Gene Ontology IDs"].split(";")))
    sequence = row["Sequence"]
    for term in terms:
        try:
            annotation = term_name_dict[term]
            dataset["Annotation"].append(annotation)
            dataset["Sequence"].append(sequence)
        except:
            pass

  0%|          | 0/11764 [00:00<?, ?it/s]

In [31]:
dataset_df = pd.DataFrame(dataset)

dataset_df.head()

,Sequence,Annotation
0,MFRRLTFAQLLFATVLGIAGGVYIFQPVFEQYAKDQKELKEKMQLV...,cellular_component mitochondrial outer membrane
1,MFRRLTFAQLLFATVLGIAGGVYIFQPVFEQYAKDQKELKEKMQLV...,biological_process response to unfolded protein
2,MFRRLTFAQLLFATVLGIAGGVYIFQPVFEQYAKDQKELKEKMQLV...,biological_process regulation of endoplasmic r...
3,MRWQEMGYIFYPRKLR,biological_process osteoblast differentiation
4,MRWQEMGYIFYPRKLR,molecular_function DNA binding


In [34]:
dataset_df.shape

(147431, 2)

In [35]:
dataset_df.to_csv("../data/protein_go_annotation.tsv", sep="\t", index=False)

Dataset({
    features: ['Sequence', 'Annotation'],
    num_rows: 147431
})